<a href="https://colab.research.google.com/github/bhattacharjee/mtu-nlp-assignment/blob/main/assignment2/notebooks/MTU_NLP_Assignment_2_Question_Answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==1.7.1        -q
!pip install tqdm==4.49.0        -q
!pip install transformers==4.2.2 -q
!pip install tensorflow          -q
!pip install tensorboard         -q
!pip install tensorboardX        -q